In [ ]:
cd ..

In [ ]:
import copy
import datetime
import os
import warnings

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
import echofilter.plotting
import echofilter.raw

In [ ]:
top_color = "c"
bot_color = "#00ee00"

In [ ]:
root_data_dir = "/data/dsforce/surveyExports"

In [ ]:
import scipy.interpolate
import scipy.ndimage

from echofilter.raw import loader, utils

In [ ]:
from echofilter.raw.manipulate import find_passive_data

In [ ]:
ROOT_DATA_DIR = loader.ROOT_DATA_DIR

In [ ]:
def find_passive_data_v2(
    signals,
    n_depth_use=38,
    threshold_inner=None,
    threshold_init=None,
    deviation=None,
    sigma_depth=0,
    sigma_time=1,
):
    """
    Find segments of Sv recording which correspond to passive recording.

    Parameters
    ----------
    signals : array_like
        Two-dimensional array of Sv values, shaped `[timestamps, depths]`.
    n_depth_use : int, optional
        How many Sv depths to use, starting with the first depths (closest
        to the sounder device). If `None` all depths are used. Default is `38`.
        The median is taken across the depths, after taking the temporal
        derivative.
    threshold_inner : float, optional
        Theshold to apply to the temporal derivative of the signal when
        detected fine-tuned start/end of passive regions.
        Default behaviour is to use a threshold automatically determined using
        `deviation` if it is set, and otherwise use a threshold of `35.0`.
    threshold_init : float, optional
        Theshold to apply during the initial scan of the start/end of passive
        regions, which seeds the fine-tuning search.
        Default behaviour is to use a threshold automatically determined using
        `deviation` if it is set, and otherwise use a threshold of `12.0`.
    deviation : float, optional
        Set `threshold_inner` to be `deviation` times the standard deviation of
        the temporal derivative of the signal. The standard deviation is
        robustly estimated based on the interquartile range.
        If this is set, `threshold_inner` must not be `None`.
        Default is `None`
    sigma_depth : float, optional
        Width of kernel for filtering signals across second dimension (depth).
        Default is `0` (no filter).
    sigma_time : float, optional
        Width of kernel for filtering signals across second dimension (time).
        Default is `1`. Set to `0` to not filter.

    Returns
    -------
    passive_start : numpy.ndarray
        Indices of rows of `signals` at which passive segments start.
    passive_end : numpy.ndarray
        Indices of rows of `signals` at which passive segments end.

    Notes
    -----
    Works by looking at the difference between consecutive recordings and
    finding large deviations.
    """
    # Ensure signals is numpy array
    signals = np.asarray(signals)

    if n_depth_use is None:
        n_depth_use = signals.shape[1]

    if sigma_depth > 0:
        signals_smooth = scipy.ndimage.gaussian_filter1d(
            signals.astype(np.float32), sigma_depth, axis=-1
        )
    else:
        signals_smooth = signals

    md_inner = np.median(np.diff(signals_smooth[:, :n_depth_use], axis=0), axis=1)

    if sigma_time > 0:
        signals_init = scipy.ndimage.gaussian_filter1d(
            signals_smooth.astype(np.float32), sigma_time, axis=0
        )
        md_init = np.median(np.diff(signals_init[:, :n_depth_use], axis=0), axis=1)
    else:
        signals_init = signals
        md_init = md_inner

    if threshold_inner is not None and deviation is not None:
        raise ValueError("Only one of `threshold_inner` and `deviation` should be set.")
    if threshold_init is None:
        if deviation is None:
            threshold_init = 12.0
        else:
            threshold_inner = (
                (np.percentile(md_init, 75) - np.percentile(md_init, 25))
                / 1.35
                * deviation
            )
    if threshold_inner is None:
        if deviation is None:
            threshold_inner = 35.0
        else:
            threshold_inner = (
                (np.percentile(md_inner, 75) - np.percentile(md_inner, 25))
                / 1.35
                * deviation
            )

    threshold_high_inner = threshold_inner
    # threshold_low_inner = -threshold_inner
    threshold_high_init = threshold_init
    threshold_low_init = -threshold_init
    indices_possible_start_init = np.nonzero(md_init < threshold_low_init)[0]
    indices_possible_end_init = np.nonzero(md_init > threshold_high_init)[0]

    if len(indices_possible_start_init) == 0 and len(indices_possible_end_init) == 0:
        return np.array([]), np.array([])

    # Fine tune indices without smoothing
    indices_possible_start = []
    indices_possible_end = []

    capture_start = None
    for i, index_p in enumerate(indices_possible_start_init):
        if capture_start is None:
            capture_start = index_p
        if (
            i + 1 >= len(indices_possible_start_init)
            or indices_possible_start_init[i + 1] > index_p + 3
        ):
            # break capture
            capture_end = index_p
            capture = np.arange(capture_start, capture_end + 1)
            indices_possible_start.append(capture[np.argmin(md_init[capture])])
            capture_start = None

    capture_start = None
    for i, index_p in enumerate(indices_possible_end_init):
        if capture_start is None:
            capture_start = index_p
        if (
            i + 1 >= len(indices_possible_end_init)
            or indices_possible_end_init[i + 1] > index_p + 3
        ):
            # break capture
            capture_end = index_p
            capture = np.arange(capture_start, capture_end + 1)
            indices_possible_end.append(capture[np.argmax(md_init[capture])])
            capture_start = None

    indices_possible_start = np.array(indices_possible_start)
    indices_possible_end = np.array(indices_possible_end)

    current_index = 0
    indices_passive_start = []
    indices_passive_end = []

    if len(indices_possible_start) > 0:
        indices_possible_start += 1

    if len(indices_possible_end) > 0:
        indices_possible_end += 1

    if len(indices_possible_end) > 0 and (
        len(indices_possible_start) == 0
        or indices_possible_end[0] < indices_possible_start[0]
    ):
        indices_passive_start.append(0)
        current_index = indices_possible_end[0]
        indices_passive_end.append(current_index)
        indices_possible_start = indices_possible_start[
            indices_possible_start > current_index
        ]
        indices_possible_end = indices_possible_end[
            indices_possible_end > current_index
        ]

    while len(indices_possible_start) > 0:
        current_index = indices_possible_start[0]
        indices_passive_start.append(current_index)
        baseline_index = max(0, current_index - 2)
        baseline = signals[baseline_index, :n_depth_use]

        # Find first column which returns to the baseline value seen before passive region
        offsets = np.nonzero(
            np.median(baseline - signals[current_index:, :n_depth_use], axis=1)
            < threshold_high_inner
        )[0]
        if len(offsets) == 0:
            current_index = signals.shape[0]
        else:
            current_index += offsets[0]
        indices_passive_end.append(current_index)

        # Remove preceding indices from the list of candidates
        indices_possible_start = indices_possible_start[
            indices_possible_start > current_index
        ]
        indices_possible_end = indices_possible_end[
            indices_possible_end > current_index
        ]

        # Check the start was sufficiently inclusive
        if current_index < signals.shape[0]:
            baseline_index = min(signals.shape[0] - 1, current_index + 1)
            baseline = signals[baseline_index, :n_depth_use]
            nonpassives = np.nonzero(
                np.median(baseline - signals[:current_index, :n_depth_use], axis=1)
                < threshold_high_inner
            )[0]
            if len(nonpassives) == 0:
                indices_passive_start[-1] = 0
            else:
                indices_passive_start[-1] = min(
                    indices_passive_start[-1],
                    nonpassives[-1] + 1,
                )

        # Combine with preceding passive segments if they overlap
        while (
            len(indices_passive_start) > 1
            and indices_passive_start[-1] <= indices_passive_end[-2]
        ):
            indices_passive_start = indices_passive_start[:-1]
            indices_passive_end = indices_passive_end[:-2] + indices_passive_end[-1:]

    return np.array(indices_passive_start), np.array(indices_passive_end)

In [ ]:
sample_paths = []
for dataset in ["MinasPassage", "GrandPassage", "mobile"]:
    for partition in ["train", "validate", "test"]:
        sample_paths += [
            os.path.join(dataset, pth)
            for pth in loader.get_partition_list(partition, dataset=dataset)
        ]
sample_paths = sorted(sample_paths)

In [ ]:
sample_paths

In [ ]:
print("Begin running on {} paths\n".format(len(sample_paths)))

for i_sample, sample_path in enumerate(sample_paths):

    print(
        "{:4d}/{:4d} {}".format(
            i_sample + 1,
            len(sample_paths),
            datetime.datetime.now().strftime("%A, %B %d, %Y at %H:%M:%S"),
        )
    )
    print(sample_path)

    fname_raw = os.path.join(root_data_dir, sample_path + "_Sv_raw.csv")
    ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(
        fname_raw, warn_row_overflow=0
    )
    is_upward_facing = depths_raw[-1] < depths_raw[0]

    nt = len(ts_raw)
    print("length: {}".format(nt))
    its_raw = np.arange(len(ts_raw))

    if "december2017" in sample_path:
        psx = np.array([])
        pex = np.array([])
    elif "march2018" in sample_path:
        psx = np.arange(0, nt, 360)
        pex = psx + 60
    elif "september2018" in sample_path:
        psx = np.arange(300, nt, 360)
        pex = psx + 60
    elif "GrandPassage" in sample_path:
        psx = np.array([0, 3120, 6540, 9960, 13380])
        psx = psx[psx < nt]
        pex = np.r_[120, psx[1:] + 420]
        pex = np.minimum(pex, nt)
    else:
        psx = None
        pex = None

    def tidy_up_line(t, d):
        if d is None:
            return np.nan * np.ones_like(ts_raw)
        is_usable = np.isfinite(d)
        if np.sum(is_usable) > 0:
            t = t[is_usable]
            d = d[is_usable]
        return np.interp(ts_raw, t, d)

    ps1, pe1 = find_passive_data(signals_raw)
    ps2, pe2 = find_passive_data_v2(signals_raw)

    fname_surface = os.path.join(root_data_dir, sample_path + "_surface.evl")

    if os.path.isfile(fname_surface):
        t_surface, d_surface = loader.evl_loader(fname_surface)
    elif is_upward_facing:
        print(
            "ERROR: Expected {} to exist when transect is upfacing.".format(
                fname_surface
            )
        )
        t_surface = ts_raw
        d_surface = np.zeros_like(ts_raw)
    else:
        # Default surface depth of 0m for downward facing data
        t_surface = ts_raw
        d_surface = np.zeros_like(ts_raw)

    # Find location of passive data.
    # Try to determine passive data as whenever the surface line is undefined.
    d_surface[np.isclose(d_surface, -10000.99)] = np.nan
    is_passive = np.isnan(d_surface)
    ps3, pe3 = echofilter.utils.get_indicator_onoffsets(is_passive > 0.5)
    ps3 = np.asarray(ps3)
    pe3 = np.asarray(pe3) + 1
    pl3 = pe3 - ps3
    li = pl3 >= 3
    ps3 = ps3[li]
    pe3 = pe3[li]
    if np.sum(~li) > 0:
        print("popped {} from v3, with lengths {}".format(np.sum(~li), pl3[~li]))

    print("starts:")
    print("xp:", psx)
    print("v1:", ps1)
    print("v2:", ps2)
    print("v3:", ps3)
    print("ends:")
    print("xp:", pex)
    print("v1:", pe1)
    print("v2:", pe2)
    print("v3:", pe3)
    print("durations:")
    if pex is not None:
        print("xp:", pex - psx)
    print("v1:", pe1 - ps1)
    print("v2:", pe2 - ps2)
    if ps3 is not None:
        print("v3:", pe3 - ps3)
    print("")

    if (
        len(ps1) != len(ps2)
        or len(pe1) != len(pe2)
        or not np.allclose(ps1, ps2)
        or not np.allclose(pe1, pe2)
    ):
        print("Warning: DIFFERENT PASSIVE PREDICTIONS v1/v2")
    if (
        ps3 is not None
        and pe3 is not None
        and (
            len(ps3) != len(ps2)
            or len(pe3) != len(pe2)
            or not np.allclose(ps3, ps2)
            or not np.allclose(pe3, pe2)
        )
    ):
        print("Warning: DIFFERENT PASSIVE PREDICTIONS v2/v3")
    if (
        psx is not None
        and pex is not None
        and (
            len(psx) != len(ps1)
            or len(pex) != len(pe1)
            or not np.allclose(psx, ps1)
            or not np.allclose(pex, pe1)
        )
    ):
        print("Warning: PASSIVE v1 DISAGREES WITH EXPECTED")
    if (
        psx is not None
        and pex is not None
        and (
            len(psx) != len(ps2)
            or len(pex) != len(pe2)
            or not np.allclose(psx, ps2)
            or not np.allclose(pex, pe2)
        )
    ):
        print("Warning: PASSIVE v2 DISAGREES WITH EXPECTED")
    if (
        ps3 is not None
        and pe3 is not None
        and psx is not None
        and pex is not None
        and (
            len(psx) != len(ps3)
            or len(pex) != len(pe3)
            or not np.allclose(psx, ps3)
            or not np.allclose(pex, pe3)
        )
    ):
        print("Warning: PASSIVE v3 DISAGREES WITH EXPECTED")

    print("")

In [ ]:
sample_paths = []
dataset = "mobile"
sample_paths = [
    os.path.join(dataset, pth)
    for pth in loader.get_partition_list("leaveout", dataset=dataset)
]
sample_paths = sorted(sample_paths)


print("Begin running on {} paths\n".format(len(sample_paths)))

for i_sample, sample_path in enumerate(sample_paths):

    print(
        "{:4d}/{:4d} {}".format(
            i_sample + 1,
            len(sample_paths),
            datetime.datetime.now().strftime("%A, %B %d, %Y at %H:%M:%S"),
        )
    )
    print(sample_path)

    fname_raw = os.path.join(root_data_dir, sample_path + "_Sv_raw.csv")
    ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(
        fname_raw, warn_row_overflow=0
    )
    is_upward_facing = depths_raw[-1] < depths_raw[0]

    nt = len(ts_raw)
    print("length: {}".format(nt))
    its_raw = np.arange(len(ts_raw))

    if "december2017" in sample_path:
        psx = np.array([])
        pex = np.array([])
    elif "march2018" in sample_path:
        psx = np.arange(0, nt, 360)
        pex = psx + 60
    elif "september2018" in sample_path:
        psx = np.arange(300, nt, 360)
        pex = psx + 60
    elif "GrandPassage" in sample_path:
        psx = np.array([0, 3120, 6540, 9960, 13380])
        psx = psx[psx < nt]
        pex = np.r_[120, psx[1:] + 420]
        pex = np.minimum(pex, nt)
    else:
        psx = None
        pex = None

    ps1, pe1 = find_passive_data(signals_raw)
    ps2, pe2 = find_passive_data_v2(signals_raw)

    fname_surface = os.path.join(root_data_dir, sample_path + "_surface.evl")

    if os.path.isfile(fname_surface):
        t_surface, d_surface = loader.evl_loader(fname_surface)
    elif is_upward_facing:
        print(
            "ERROR: Expected {} to exist when transect is upfacing.".format(
                fname_surface
            )
        )
        t_surface = ts_raw
        d_surface = np.zeros_like(ts_raw)
    else:
        # Default surface depth of 0m for downward facing data
        t_surface = ts_raw
        d_surface = np.zeros_like(ts_raw)

    # Find location of passive data.
    # Try to determine passive data as whenever the surface line is undefined.
    d_surface[np.isclose(d_surface, -10000.99)] = np.nan
    is_passive = np.isnan(d_surface)
    ps3, pe3 = echofilter.utils.get_indicator_onoffsets(is_passive > 0.5)
    ps3 = np.asarray(ps3)
    pe3 = np.asarray(pe3) + 1
    pl3 = pe3 - ps3
    li = pl3 >= 3
    ps3 = ps3[li]
    pe3 = pe3[li]
    if np.sum(~li) > 0:
        print("popped {} from v3, with lengths {}".format(np.sum(~li), pl3[~li]))

    print("starts:")
    print("xp:", psx)
    print("v1:", ps1)
    print("v2:", ps2)
    print("v3:", ps3)
    print("ends:")
    print("xp:", pex)
    print("v1:", pe1)
    print("v2:", pe2)
    print("v3:", pe3)
    print("durations:")
    if pex is not None:
        print("xp:", pex - psx)
    print("v1:", pe1 - ps1)
    print("v2:", pe2 - ps2)
    if ps3 is not None:
        print("v3:", pe3 - ps3)
    print("")

    if (
        len(ps1) != len(ps2)
        or len(pe1) != len(pe2)
        or not np.allclose(ps1, ps2)
        or not np.allclose(pe1, pe2)
    ):
        print("Warning: DIFFERENT PASSIVE PREDICTIONS v1/v2")
    if (
        ps3 is not None
        and pe3 is not None
        and (
            len(ps3) != len(ps2)
            or len(pe3) != len(pe2)
            or not np.allclose(ps3, ps2)
            or not np.allclose(pe3, pe2)
        )
    ):
        print("Warning: DIFFERENT PASSIVE PREDICTIONS v2/v3")
    if (
        psx is not None
        and pex is not None
        and (
            len(psx) != len(ps1)
            or len(pex) != len(pe1)
            or not np.allclose(psx, ps1)
            or not np.allclose(pex, pe1)
        )
    ):
        print("Warning: PASSIVE v1 DISAGREES WITH EXPECTED")
    if (
        psx is not None
        and pex is not None
        and (
            len(psx) != len(ps2)
            or len(pex) != len(pe2)
            or not np.allclose(psx, ps2)
            or not np.allclose(pex, pe2)
        )
    ):
        print("Warning: PASSIVE v2 DISAGREES WITH EXPECTED")
    if (
        ps3 is not None
        and pe3 is not None
        and psx is not None
        and pex is not None
        and (
            len(psx) != len(ps3)
            or len(pex) != len(pe3)
            or not np.allclose(psx, ps3)
            or not np.allclose(pex, pe3)
        )
    ):
        print("Warning: PASSIVE v3 DISAGREES WITH EXPECTED")

    print("")

In [ ]:
bad_sample_paths = [
    "MinasPassage/september2018/september2018_D20181021-T165220_D20181021-T222221",
    "MinasPassage/september2018/september2018_D20181022-T105220_D20181022-T162217",
    "MinasPassage/september2018/september2018_D20181022-T172213_D20181022-T232217",
    "MinasPassage/september2018/september2018_D20181026-T082220_D20181026-T135213",
    "MinasPassage/september2018/september2018_D20181026-T142217_D20181026-T195218",
]
# bad_sample_paths = [
# "MinasPassage/september2018/september2018_D20180928-T202217_D20180929-T015217",
# "MinasPassage/september2018/september2018_D20181008-T235218_D20181009-T052220",
# "MinasPassage/september2018/september2018_D20181021-T045220_D20181021-T102218",
# ]
# bad_sample_paths = [
#    "GrandPassage/phase2/GrandPassage_WBAT_2B_20200130_UTC020017_floodhigh",
# ]

In [ ]:
print("Begin running on {} paths\n".format(len(bad_sample_paths)))

for i_sample, sample_path in enumerate(bad_sample_paths):

    print(
        "{:4d}/{:4d} {}".format(
            i_sample + 1,
            len(bad_sample_paths),
            datetime.datetime.now().strftime("%A, %B %d, %Y at %H:%M:%S"),
        )
    )
    print(sample_path)

    fname_raw = os.path.join(root_data_dir, sample_path + "_Sv_raw.csv")
    ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(
        fname_raw, warn_row_overflow=0
    )
    is_upward_facing = depths_raw[-1] < depths_raw[0]

    nt = len(ts_raw)
    print("length: {}".format(nt))
    its_raw = np.arange(len(ts_raw))

    if "december2017" in sample_path:
        psx = np.array([])
        pex = np.array([])
    elif "march2018" in sample_path:
        psx = np.arange(0, nt, 360)
        pex = psx + 60
    elif "september2018" in sample_path:
        psx = np.arange(300, nt, 360)
        pex = psx + 60
    elif "GrandPassage" in sample_path:
        psx = np.array([0, 3120, 6540, 9960, 13380])
        psx = psx[psx < nt]
        pex = np.r_[120, psx[1:] + 420]
        pex = np.minimum(pex, nt)
    else:
        psx = None
        pex = None

    ps1, pe1 = find_passive_data(signals_raw)
    ps2, pe2 = find_passive_data_v2(signals_raw)

    fname_surface = os.path.join(root_data_dir, sample_path + "_surface.evl")

    if os.path.isfile(fname_surface):
        t_surface, d_surface = loader.evl_loader(fname_surface)
    elif is_upward_facing:
        print(
            "ERROR: Expected {} to exist when transect is upfacing.".format(
                fname_surface
            )
        )
        t_surface = ts_raw
        d_surface = np.zeros_like(ts_raw)
    else:
        # Default surface depth of 0m for downward facing data
        t_surface = ts_raw
        d_surface = np.zeros_like(ts_raw)

    # Find location of passive data.
    # Try to determine passive data as whenever the surface line is undefined.
    d_surface[np.isclose(d_surface, -10000.99)] = np.nan
    is_passive = np.isnan(d_surface)
    ps3, pe3 = echofilter.utils.get_indicator_onoffsets(is_passive > 0.5)
    ps3 = np.asarray(ps3)
    pe3 = np.asarray(pe3) + 1
    pl3 = pe3 - ps3
    li = pl3 >= 3
    ps3 = ps3[li]
    pe3 = pe3[li]
    if np.sum(~li) > 0:
        print("popped {} from v3, with lengths {}".format(np.sum(~li), pl3[~li]))

    print("starts:")
    print("xp:", psx)
    print("v1:", ps1)
    print("v2:", ps2)
    print("v3:", ps3)
    print("ends:")
    print("xp:", pex)
    print("v1:", pe1)
    print("v2:", pe2)
    print("v3:", pe3)
    print("durations:")
    if pex is not None:
        print("xp:", pex - psx)
    print("v1:", pe1 - ps1)
    print("v2:", pe2 - ps2)
    if ps3 is not None:
        print("v3:", pe3 - ps3)
    print("")

    if (
        len(ps1) != len(ps2)
        or len(pe1) != len(pe2)
        or not np.allclose(ps1, ps2)
        or not np.allclose(pe1, pe2)
    ):
        print("Warning: DIFFERENT PASSIVE PREDICTIONS v1/v2")
    if (
        ps3 is not None
        and pe3 is not None
        and (
            len(ps3) != len(ps2)
            or len(pe3) != len(pe2)
            or not np.allclose(ps3, ps2)
            or not np.allclose(pe3, pe2)
        )
    ):
        print("Warning: DIFFERENT PASSIVE PREDICTIONS v2/v3")
    if (
        psx is not None
        and pex is not None
        and (
            len(psx) != len(ps1)
            or len(pex) != len(pe1)
            or not np.allclose(psx, ps1)
            or not np.allclose(pex, pe1)
        )
    ):
        print("Warning: PASSIVE v1 DISAGREES WITH EXPECTED")
    if (
        psx is not None
        and pex is not None
        and (
            len(psx) != len(ps2)
            or len(pex) != len(pe2)
            or not np.allclose(psx, ps2)
            or not np.allclose(pex, pe2)
        )
    ):
        print("Warning: PASSIVE v2 DISAGREES WITH EXPECTED")
    if (
        ps3 is not None
        and pe3 is not None
        and psx is not None
        and pex is not None
        and (
            len(psx) != len(ps3)
            or len(pex) != len(pe3)
            or not np.allclose(psx, ps3)
            or not np.allclose(pex, pe3)
        )
    ):
        print("Warning: PASSIVE v3 DISAGREES WITH EXPECTED")

    best_ps = ps1
    best_pe = pe1

    for i in range(min(len(best_ps), len(psx))):
        if best_ps[i] == psx[i] and best_pe[i] == pex[i]:
            continue

        for ps, pe, tit in (
            (psx[i], pex[i], "expected"),
            (best_ps[i], best_pe[i], "v1"),
        ):
            plt.figure(figsize=(12, 9))
            i0 = max(0, ps - 1)
            i1 = pe + 2
            if i1 >= len(its_raw):
                i1 = None
            plt.pcolormesh(its_raw[i0:i1], depths_raw[:50], signals_raw[i0:i1, :50].T)
            plt.gca().invert_yaxis()
            plt.title("passive #{}, {}".format(i, tit))
            plt.show()

    print("")

In [ ]:
plt.figure(figsize=(12, 9))
plt.pcolormesh(its_raw, depths_raw, signals_raw.T)
plt.gca().invert_yaxis()
plt.title(sample_path)
plt.show()

In [ ]:
_ = """
length: 10259
starts:
xp: [   0 3120 6540 9960]
v1: [   0 3120 6539 9959]
v2: [   0 3120 6539 9959]
v3: [   0 3120 6540 9960]
ends:
xp: [  120  3540  6960 10259]
v1: [  120  3540  6959 10259]
v2: [  120  3540  6701 10259]
v3: [  120  3540  6960 10260]
durations:
xp: [120 420 420 299]
v1: [120 420 420 300]
v2: [120 420 162 300]
v3: [120 420 420 300]
"""

In [ ]:
i0 = 10200
i1 = 10259
i0 -= 1
i1 += 2
plt.figure(figsize=(12, 9))
plt.pcolormesh(its_raw[i0:i1], depths_raw[:50], signals_raw[i0:i1, :50].T)
plt.gca().invert_yaxis()
plt.title("{}-{}".format(i0, i1 - 1))
plt.show()

In [ ]:
sample_path = (
    "MinasPassage/december2017/december2017_D20180222-T145219_D20180222-T142214"
)

fname_raw = os.path.join(root_data_dir, sample_path + "_Sv_raw.csv")
ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(
    fname_raw, warn_row_overflow=0
)
is_upward_facing = depths_raw[-1] < depths_raw[0]

nt = len(ts_raw)
print("length: {}".format(nt))
its_raw = np.arange(len(ts_raw))

In [ ]:
plt.figure(figsize=(12, 9))
plt.pcolormesh(its_raw, depths_raw, signals_raw.T)
plt.gca().invert_yaxis()
plt.title(sample_path)
plt.show()

In [ ]:
i0 = 250
i1 = 350
i0 -= 1
i1 += 2
plt.figure(figsize=(12, 9))
plt.pcolormesh(its_raw[i0:i1], depths_raw[:50], signals_raw[i0:i1, :50].T)
plt.gca().invert_yaxis()
plt.title("{}-{}".format(i0, i1 - 1))
plt.show()

In [ ]:
bad_sample_paths = [
    "mobile/Survey01/Survey01_GR1_S1A_E",
    "mobile/Survey03/Survey03_GR2_N5W_survey3",
    "mobile/Survey03/Survey03_GR4_N0A_survey3",
    "mobile/Survey04/Survey04_GR1_N3A",
    "mobile/Survey04/Survey04_GR2_N5A",
    "mobile/Survey05/Survey05_GR1_N1A_survey5",
    "mobile/Survey07/Survey07_GR2_N1W_survey7",
    "mobile/Survey10/Survey10_GR1_N0A_E",
    "mobile/Survey12/Survey12_GR4_N5A_E",
    "mobile/Survey01/Survey01_GR1_S2A_E",
    "mobile/Survey01/Survey01_GR1_S2W_E",
    "mobile/Survey11/Survey11_GR1_S2A_E",
]

In [ ]:
print("Begin running on {} paths\n".format(len(bad_sample_paths)))

for i_sample, sample_path in enumerate(bad_sample_paths):

    print(
        "{:4d}/{:4d} {}".format(
            i_sample + 1,
            len(bad_sample_paths),
            datetime.datetime.now().strftime("%A, %B %d, %Y at %H:%M:%S"),
        )
    )
    print(sample_path)

    fname_raw = os.path.join(root_data_dir, sample_path + "_Sv_raw.csv")
    ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(
        fname_raw, warn_row_overflow=0
    )
    is_upward_facing = depths_raw[-1] < depths_raw[0]

    nt = len(ts_raw)
    print("length: {}".format(nt))
    its_raw = np.arange(len(ts_raw))

    plt.figure(figsize=(12, 9))
    plt.pcolormesh(its_raw, depths_raw, signals_raw.T)
    plt.gca().invert_yaxis()
    plt.title(sample_path)
    plt.show()

    print("")